In [1]:
# Initial Library imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np
import datetime as dt


In [2]:
# Load .env environment variables
load_dotenv()

True

In [3]:
# Set adjustable variables
no_tickers = 30
start_date = "2022-05-10" # format yyyy-mm-dd
end_date   = "2022-05-17" # format yyyy-mm-dd *** note with 4Hour timeframe, data will capture up to end of previous day from end_date

#Set timeframe of Tickers 
timeframe = "4Hour"

In [4]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [5]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [6]:
# import csv list of all nasdaq tickers and clean them for use

#import csv file of all tickers to a dataframe
df_full_ticker_list = pd.read_csv ('Data/nasdaq_screener_assets.csv')
#drop unwanted columns
df_full_ticker_list = df_full_ticker_list.drop(['Last Sale', 'Net Change', '% Change', 'Market Cap',
       'Country', 'IPO Year', 'Volume', 'Sector', 'Industry'], axis = 'columns')

# Remove Tickers with '^', '/', & spaces as Alpaca does not accept them
df_full_ticker_list = df_full_ticker_list[~df_full_ticker_list.Symbol.str.contains('^', regex = False)
                                    & ~df_full_ticker_list.Symbol.str.contains('/', regex = False)
                                    & ~df_full_ticker_list.Symbol.str.contains(' ', regex = False)
                                   ]

# convert df to list for use with Alpaca API
alpaca_tickers = list(df_full_ticker_list['Symbol'])


In [7]:
#set up for Alpaca API call

# Format current date as ISO format
start_date = pd.Timestamp(start_date, tz="America/New_York").isoformat()
end_date = pd.Timestamp(end_date, tz="America/New_York").isoformat()


In [8]:
# make API call to Alpaca to receive a data frame of all stock data

df_stock_all = alpaca.get_bars(
    alpaca_tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

#check output of df_stock_all
df_stock_all.head(10)

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2022-05-10 08:00:00+00:00,55.600,55.600,54.60,55.030,34898,288,55.157951,AA
2022-05-10 12:00:00+00:00,54.900,56.880,54.07,54.520,4036281,38858,55.486529,AA
2022-05-10 16:00:00+00:00,54.580,56.910,53.61,55.835,4177742,50011,55.568406,AA
2022-05-10 20:00:00+00:00,55.840,56.100,55.61,55.700,203721,27,55.839540,AA
2022-05-11 08:00:00+00:00,56.890,56.930,56.50,56.930,955,17,56.761602,AA
2022-05-11 12:00:00+00:00,57.000,59.360,55.19,58.370,2748562,30562,58.218161,AA
2022-05-11 16:00:00+00:00,58.405,58.405,56.27,56.620,3124526,32199,57.113865,AA
2022-05-11 20:00:00+00:00,56.640,56.790,56.00,56.660,229478,105,56.648243,AA
2022-05-12 08:00:00+00:00,56.000,56.000,54.31,55.150,4834,50,54.991585,AA


In [9]:
# calculate the change in price accross the selected date range to determine a suitable dataset for analysis

# Set up df_stock_price_change varialbe as data frame to capture all start and end values
df_stock_price_change = pd.DataFrame(columns = ['Ticker', 'Start Price', 'End Price','Price Change'])
# capture all Ticker symbols in df_stock_price_change form df_full_ticker_list
df_stock_price_change['Ticker'] = df_full_ticker_list['Symbol']

# loop through each ticker and capture the first and last pice for each
for index in df_stock_price_change.index:
    ticker = df_stock_price_change['Ticker'][index]
    ticker_data_temp = df_stock_all[df_stock_all.symbol == ticker]
    
    # ignore ticker if no symbol was found from Alpaca, only process if the size of ticker_data_temp is greater than 0
    if ticker_data_temp.size != 0:
    
        df_stock_price_change['Start Price'][index] = ticker_data_temp.at[ticker_data_temp.index.max(), 'close']
        df_stock_price_change['End Price'][index] = ticker_data_temp.at[ticker_data_temp.index.min(), 'close']

# calculate price change
df_stock_price_change['Price Change'] = df_stock_price_change['End Price'] - df_stock_price_change['Start Price']
df_stock_price_change['Price Change %'] = (df_stock_price_change['Price Change']/df_stock_price_change['Start Price'])*100

# check output of df_stock_price_change
df_stock_price_change.head(10)

,Ticker,Start Price,End Price,Price Change,Price Change %
0,A,120.3,114.46,-5.84,-4.85453
1,AA,61.11,55.03,-6.08,-9.949272
2,AAC,9.79,9.815,0.025,0.255363
3,AACG,1.01,1.09,0.08,7.920792
4,AACI,9.84,9.84,0.0,0.0
5,AACIW,0.2678,0.2698,0.002,0.746826
6,AADI,13.72,12.67,-1.05,-7.653061
7,AAIC,3.62,3.0482,-0.5718,-15.79558
10,AAIN,24.19,23.245,-0.945,-3.906573
11,AAL,16.87,16.53,-0.34,-2.015412


In [10]:
# Clean df_stock_price_change, drop NaNs and sort in order of % change
df_stock_price_change.sort_values(by = 'Price Change %',ascending = False, inplace = True)
df_stock_price_change.dropna(axis = 'index', how = 'any' ,inplace = True)
df_stock_price_change.reset_index(drop = True, inplace = True)

# check output of df_stock_price_change
df_stock_price_change.head(10)

,Ticker,Start Price,End Price,Price Change,Price Change %
0,TNON,5.85,50.0,44.15,754.700855
1,RDBXW,0.104,0.5,0.396,380.769231
2,CMRX,1.63,4.155,2.525,154.907975
3,SRSAW,0.0895,0.2233,0.1338,149.497207
4,ITP,0.1691,0.3895,0.2204,130.337079
5,VIEW,0.6059,1.36,0.7541,124.459482
6,LIQT,0.523,1.16,0.637,121.797323
7,GSAQW,0.0501,0.105,0.0549,109.580838
8,IMAQW,0.0681,0.1425,0.0744,109.251101
9,DMS,1.46,2.92,1.46,100.0


In [11]:
# Select Tickers for analysis distributed through Data Set equally
# this is to ensure that the tickers selected for analysis are a well represented set across a wide market range 

#set number of rows
rows_count = df_stock_price_change.shape[0]
# calculate step size for selection less one to avoid the ends
increment = round(rows_count/(no_tickers))-1
# set indexes of tickers to capture for analysis
list_select_tickers = range(round(no_tickers/2), rows_count, increment)

# capture the ticker symbols to use for anlysis
list_analysis_tickers = list(df_stock_price_change['Ticker'][list_select_tickers].values)

# check output of list_analysis_tickers
list_analysis_tickers

['POAI',
 'ACLX',
 'SBNY',
 'COSM',
 'SQ',
 'HGEN',
 'KWR',
 'FCT',
 'GRC',
 'MAR',
 'MPRAU',
 'HORI',
 'OEPWU',
 'IVCB',
 'AACI',
 'CREC',
 'BKCC',
 'SBTX',
 'IMH',
 'WEX',
 'PWR',
 'WNC',
 'GBX',
 'CGABL',
 'BSM',
 'DOCU',
 'DRQ',
 'TRIP',
 'INSW',
 'CBRGW',
 'SHCRW']

In [12]:
# capture stock price data of the selected tickers and clean ready for analysis

# capture the data for only the chosen tickers from df_stock_all
df_stock_price_data = df_stock_all[df_stock_all['symbol'].isin(list_analysis_tickers)]
# reset index
df_stock_price_data.reset_index(inplace = True)

# Change 'timestamp' values to date only
#df_stock_price_data.loc[:,'timestamp'] = df_stock_price_data.loc[:,'timestamp'].dt.date

# drop un-needed columns
df_stock_price_data.drop(['open','high','low','volume','trade_count','vwap'], axis='columns',inplace = True)
# rename remaining columns to suitable names
df_stock_price_data.columns = ['Date','Close','Ticker']

# check output of df_stock_price_data
df_stock_price_data.head(10)

C:\Users\mclew\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Date,Close,Ticker
0,2022-05-16 12:00:00+00:00,9.840,AACI
1,2022-05-10 12:00:00+00:00,9.495,ACLX
2,2022-05-10 16:00:00+00:00,9.620,ACLX
3,2022-05-10 20:00:00+00:00,9.690,ACLX
4,2022-05-11 12:00:00+00:00,9.260,ACLX
5,2022-05-11 16:00:00+00:00,8.855,ACLX
6,2022-05-11 20:00:00+00:00,8.890,ACLX
7,2022-05-12 12:00:00+00:00,8.950,ACLX
8,2022-05-12 16:00:00+00:00,8.840,ACLX
9,2022-05-12 20:00:00+00:00,8.800,ACLX


In [13]:
# save CSV of df_stock_price_data as 'Data/StockPriceData.csv'
df_stock_price_data.to_csv('Data/StockPriceData.csv')

In [14]:
#save ticker list in .csv for use in other program
#convert list to DF
df_alpaca_tickers = pd.DataFrame(alpaca_tickers)
# save to CSV
df_alpaca_tickers.to_csv('Data/TickerList.csv')

In [15]:
df_stock_price_data.head(10)

,Date,Close,Ticker
0,2022-05-16 12:00:00+00:00,9.840,AACI
1,2022-05-10 12:00:00+00:00,9.495,ACLX
2,2022-05-10 16:00:00+00:00,9.620,ACLX
3,2022-05-10 20:00:00+00:00,9.690,ACLX
4,2022-05-11 12:00:00+00:00,9.260,ACLX
5,2022-05-11 16:00:00+00:00,8.855,ACLX
6,2022-05-11 20:00:00+00:00,8.890,ACLX
7,2022-05-12 12:00:00+00:00,8.950,ACLX
8,2022-05-12 16:00:00+00:00,8.840,ACLX
9,2022-05-12 20:00:00+00:00,8.800,ACLX


In [16]:
df_stock_all.to_csv('Data/AllStockData.csv')